In [1]:
!pip install git+https://github.com/TransformerLensOrg/TransformerLens

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/TransformerLensOrg/TransformerLens to /tmp/pip-req-build-oyt_na_i
  Running command git clone --filter=blob:none --quiet https://github.com/TransformerLensOrg/TransformerLens /tmp/pip-req-build-oyt_na_i
  Resolved https://github.com/TransformerLensOrg/TransformerLens to commit cb5017ad0f30cde0d3ac0b0f863c27fbec964c28
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
!pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.1 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [3]:
!pip install webdataset
!pip install datasets
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python -m pip install --upgrade pip


In [4]:
import sys
from pathlib import Path
from io import BytesIO
import json

import torch as t
import numpy as np
import transformer_lens as tl
import webdataset as wds
import datasets
import nltk

In [5]:
opt_small = tl.HookedTransformer.from_pretrained('opt-125m')
gpt2_small = tl.HookedTransformer.from_pretrained('gpt2-small')

/home/terekhov/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Loaded pretrained model opt-125m into HookedTransformer
Loaded pretrained model gpt2-small into HookedTransformer


In [6]:
text = "This project will look into cross-model similarity in the hidden representations in the residual stream. The main outcomes are"
logits, cache = opt_small.run_with_cache(text)
cache.keys()

dict_keys(['hook_embed', 'hook_pos_embed', 'blocks.0.hook_resid_pre', 'blocks.0.ln1.hook_scale', 'blocks.0.ln1.hook_normalized', 'blocks.0.attn.hook_q', 'blocks.0.attn.hook_k', 'blocks.0.attn.hook_v', 'blocks.0.attn.hook_attn_scores', 'blocks.0.attn.hook_pattern', 'blocks.0.attn.hook_z', 'blocks.0.hook_attn_out', 'blocks.0.hook_resid_mid', 'blocks.0.ln2.hook_scale', 'blocks.0.ln2.hook_normalized', 'blocks.0.mlp.hook_pre', 'blocks.0.mlp.hook_post', 'blocks.0.hook_mlp_out', 'blocks.0.hook_resid_post', 'blocks.1.hook_resid_pre', 'blocks.1.ln1.hook_scale', 'blocks.1.ln1.hook_normalized', 'blocks.1.attn.hook_q', 'blocks.1.attn.hook_k', 'blocks.1.attn.hook_v', 'blocks.1.attn.hook_attn_scores', 'blocks.1.attn.hook_pattern', 'blocks.1.attn.hook_z', 'blocks.1.hook_attn_out', 'blocks.1.hook_resid_mid', 'blocks.1.ln2.hook_scale', 'blocks.1.ln2.hook_normalized', 'blocks.1.mlp.hook_pre', 'blocks.1.mlp.hook_post', 'blocks.1.hook_mlp_out', 'blocks.1.hook_resid_post', 'blocks.2.hook_resid_pre', 'block

In [7]:
print(opt_small.to_str_tokens(text))
print(gpt2_small.to_str_tokens(text))

['</s>', 'This', ' project', ' will', ' look', ' into', ' cross', '-', 'model', ' similarity', ' in', ' the', ' hidden', ' representations', ' in', ' the', ' residual', ' stream', '.', ' The', ' main', ' outcomes', ' are']
['<|endoftext|>', 'This', ' project', ' will', ' look', ' into', ' cross', '-', 'model', ' similarity', ' in', ' the', ' hidden', ' representations', ' in', ' the', ' residual', ' stream', '.', ' The', ' main', ' outcomes', ' are']


Aha, I'm guessing OPT has a very similar tokenization scheme. I can filter examples where the tokenization of the sentence is the same.

In [8]:
opt_small.W_K.shape

torch.Size([12, 12, 768, 64])

In [9]:
nlayers = opt_small.W_K.shape[0]
mid_layer = nlayers // 2
d_model = opt_small.W_K.shape[2]
cached_value = None

def cache_it(resid, hook):
    global cached_value
    cached_value = resid.detach()
    # print(hook)
    return resid

loss = opt_small.run_with_hooks(text, return_type="loss", fwd_hooks=[(f'blocks.{mid_layer}.hook_resid_post', cache_it)])

In [10]:
opt_small

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h

In [11]:
gpt2_small

HookedTransformer(
  (embed): Embed()
  (hook_embed): HookPoint()
  (pos_embed): PosEmbed()
  (hook_pos_embed): HookPoint()
  (blocks): ModuleList(
    (0-11): 12 x TransformerBlock(
      (ln1): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (ln2): LayerNormPre(
        (hook_scale): HookPoint()
        (hook_normalized): HookPoint()
      )
      (attn): Attention(
        (hook_k): HookPoint()
        (hook_q): HookPoint()
        (hook_v): HookPoint()
        (hook_z): HookPoint()
        (hook_attn_scores): HookPoint()
        (hook_pattern): HookPoint()
        (hook_result): HookPoint()
      )
      (mlp): MLP(
        (hook_pre): HookPoint()
        (hook_post): HookPoint()
      )
      (hook_attn_in): HookPoint()
      (hook_q_input): HookPoint()
      (hook_k_input): HookPoint()
      (hook_v_input): HookPoint()
      (hook_mlp_in): HookPoint()
      (hook_attn_out): HookPoint()
      (hook_mlp_out): HookPoint()
      (h

In [12]:
cached_value.reshape((-1, d_model)).shape

In [13]:
texts = ["This project will look into cross-model similarity in the hidden representations in the residual stream. The main outcomes are",
         "A powerful AI model that can generate text from a prompt. The model is trained on a large corpus of text data.",
         "This text was generated by Copilot, excluding, ironically, this line"]

writer = wds.TarWriter("../data/test.tar")

Path("test").mkdir(exist_ok=True)
for i, text in enumerate(texts):
    loss = opt_small.run_with_hooks(text, return_type="loss", fwd_hooks=[(f'blocks.{mid_layer}.hook_resid_post', cache_it)])
    writer.write({
        "__key__": f"{i:06d}",
        "text.txt": text,
        "values.npy": cached_value.cpu().numpy()
    })

writer.close()

In [14]:
dataset = wds.WebDataset("../data/test.tar")

In [15]:
for item in iter(dataset):
    print(item["text.txt"])
    # Read from an npy file
    values = np.load(BytesIO(item["values.npy"]))
    print(values.shape)

Ok now we want to prepare a large set of sentences from wikipedia.

In [16]:
wiki = datasets.load_dataset("wikipedia", "20220301.en")

In [17]:
len(wiki["train"])

In [18]:
wiki["train"][0]

In [19]:
nltk.download('punkt')
nltk.download('punkt_tab')

In [20]:
nltk.sent_tokenize(wiki["train"][0]["text"])

In [21]:
train_frac = 0.8
test_frac = 1.0 - train_frac

train_mask = np.random.rand(len(wiki["train"])) < train_frac
test_mask = ~train_mask

In [22]:
print(train_mask[:10])

In [23]:
sum(train_mask), sum(test_mask)

In [24]:
train_inds = np.where(train_mask)[0]
test_inds = np.where(test_mask)[0]

train_inds = np.random.permutation(train_inds)
test_inds = np.random.permutation(test_inds)

In [25]:
np.savez("../data/wiki_split.npz", train_inds=train_inds, test_inds=test_inds)

In [26]:
train_inds[:10]

In [27]:
from tqdm.notebook import tqdm

In [28]:
# Check a few random sentences on whether the tokenization is the same
num_samples = 10000
num_diff = 0
for i in tqdm(train_inds[:num_samples]):
    text = wiki["train"][int(i)]["text"]
    sent = nltk.sent_tokenize(text)
    random_sent = np.random.choice(sent)
    # print(random_sent)
    opt_tokens = opt_small.to_str_tokens(random_sent)
    gpt2_tokens = gpt2_small.to_str_tokens(random_sent)
    opt_tokens = opt_tokens[1:]
    gpt2_tokens = gpt2_tokens[1:]
    if opt_tokens != gpt2_tokens:
        num_diff += 1
print(f"Number of different tokenizations: {num_diff}/{num_samples}")

In [29]:
def get_middle_hook(model):
    nlayers = model.W_K.shape[0]
    mid_layer = nlayers // 2
    return f'blocks.{mid_layer}.hook_resid_post'

In [33]:
print(get_middle_hook(opt_small))
print(get_middle_hook(gpt2_small))

In [34]:
from datetime import datetime

In [32]:
num_sentences = 5
batch_size = 32
max_tokens = 1024
# num_samples = 100_000
# num_samples = 10_000
start_sample = 10_000
end_sample = 30_000
writer = wds.TarWriter("../data/wiki_sentences_30_60k.tar")

with open(f'../outputs/log_{datetime.now():%Y%m%d_%H%M%S}.txt', 'w') as log_file:
    num_sampled = 0
    num_mismatched = 0
    for ii, i in tqdm(enumerate(train_inds[start_sample:end_sample]), total=start_sample - end_sample):
        text = wiki["train"][int(i)]["text"]
        sent = nltk.sent_tokenize(text)

        print(f"{ii} before check len", file=log_file, flush=True)
        if len(sent) <= num_sentences:
            continue
        print(f"{ii} after check len", file=log_file, flush=True)
        random_chunk = np.random.randint(0, len(sent) - num_sentences)
        chunk = sent[random_chunk:random_chunk + num_sentences]
        chunk = " ".join(chunk)
        opt_tokens = opt_small.to_str_tokens(chunk)
        gpt2_tokens = gpt2_small.to_str_tokens(chunk)
        print(f"{ii} after composing the sent", file=log_file, flush=True)
        
        if len(opt_tokens) > max_tokens or len(gpt2_tokens) > max_tokens:
            continue
            
        print(f"{ii} token check", file=log_file, flush=True)
            
        if opt_tokens[1:] != gpt2_tokens[1:]:
            num_mismatched += 1

        num_sampled += 1
        
        gpt2_small.run_with_hooks(chunk, return_type="loss", fwd_hooks=[(get_middle_hook(gpt2_small), cache_it)])
        resid_gpt2 = cached_value.detach().cpu().numpy()
        print(f"{ii} gpt2", file=log_file, flush=True)
        opt_small.run_with_hooks(chunk, return_type="loss", fwd_hooks=[(get_middle_hook(opt_small), cache_it)])
        resid_opt = cached_value.detach().cpu().numpy()
        print(f"{ii} opt", file=log_file, flush=True)
        
        # print(resid_gpt2.shape, resid_opt.shape)
        
        writer.write({
            "__key__": f"{num_sampled:08d}",
            "text.txt": chunk,
            "resid_gpt2.npy": resid_gpt2,
            "resid_opt.npy": resid_opt
        })

        print(f"{ii} written", file=log_file, flush=True)
        
    writer.close()

In [71]:
# import gc
# gc.collect()
# t.cuda.empty_cache()

In [ ]:
print(f"{num_mismatched=} {num_sampled=}")